In [126]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib .pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [135]:
# Main
i = 1
trials = 1
data = extract_features()
ensamble_list = []
ensamble_list.append(create_classifier(data))
ensemble_error = compute_ensemble_error(data,ensamble_list) # Firts enbamble error
r_size = 10 # Number of instances of training data to generate
c_size = 5 # Number of classifiers in the ensamble
i_max = 10 # Number of iterations
while i < c_size and trials < i_max :
    r = data.append(create_lable_new_examples(data,ensamble_list,r_size))
    ensamble_list.append(create_classifier(r))
    ensemble_error_r = compute_ensemble_error(data,ensamble_list[-1])
    if ensemble_error_r < ensemble_error:
        i = i + 1
        ensemble_error = ensemble_error_r 
    else:
        del ensemble_error[-1]
    trials = trials + 1


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [127]:
def extract_features():
    file_names=['Credit Approval'] # will need to be removed
    for i in file_names:
        file_directory = 'Datasets/'+i+'.csv'
        data = pd.read_csv(file_directory)
        data = data.replace(['?',' '], np.nan) # replace all empty values to np.nan
        data = data.dropna(axis=0,how='any') # remove all rows with any np.nan
    for i in data.columns:
        if data[i].dtype == object:
            data[i] = pd.factorize(data[i])[0]
    return data

In [128]:
def create_classifier(data): # initialize with the Credit Approval data set. later, no such data sets (combinations of categorical 
    # and numeric values) will be used
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1:]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=101)
    dtree = DecisionTreeClassifier()
    dtree.fit(x_train,y_train)
    predictions = dtree.predict(x_test)
    #import pdb; pdb.set_trace()
    return (dtree)

In [129]:
# Compute C* and epsilon
def compute_ensemble_error(data,ensamble_list):
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1:]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.33,random_state=101)
    #import pdb; pdb.set_trace()
    comulative_error = 0 
    predictions_list = []
    for dtree in ensamble_list: # generate list if all classifiers classifications
        predictions_list.append(dtree.predict(x_test))
    c_star =pd.DataFrame(pd.DataFrame(predictions_list).T.apply(lambda x: x.value_counts(), axis = 1).idxmax(axis=1))
    #import pdb; pdb.set_trace()
    ensamble_error = confusion_matrix(y_test,c_star)
    return (ensamble_error)

In [130]:
def create_training_examples(data,r_size):
    mean = data.mean()
    std = data.std()
    dic = dict()
    for i,col in enumerate(data.columns):
        dic[col] = np.random.normal(mean[i],std[i],r_size)
    data.append(pd.DataFrame(dic), ignore_index = True)
    return data.iloc[-r:,:]

In [134]:
def create_lable_new_examples(data,ensamble_list,r_size):
    r_examples = create_training_examples(data,r_size)
    predictions = pd.DataFrame([[0]*r_size,[1]*r_size]).T
    #import pdb; pdb.set_trace()
    for i,model in enumerate(ensamble_list):
        predictions.append(pd.DataFrame(ensamble_list[i].predict(r_examples.iloc[:,:-1])),ignore_index = True)
    predictions = predictions.apply(lambda x: x.value_counts(), axis = 1).idxmin(axis=1)
    r_examples.iloc[:,-1:] = predictions
    return (r_examples)